In [29]:
%reload_ext autoreload
%autoreload 2

In [30]:
import pandas as pd
import numpy as np

In [31]:
raw_data = '../data/raw'

In [32]:
train = pd.read_csv(f'{raw_data}/train.csv', low_memory=False)
train.head(1)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1


In [33]:
y = train['Sales']

In [34]:
from sklearn.preprocessing import OneHotEncoder
train = pd.concat([train, train['StateHoliday']], axis=1)
train.drop(['StateHoliday'], axis=1, inplace=True)

In [35]:
train.drop(['Customers'], axis=1, inplace=True)

In [36]:
train['Date'] = pd.to_datetime(train['Date'])
for fld in ['year', 'week', 'day', 'dayofweek', 'dayofyear', 'weekday', 'weekofyear', 'quarter']:
    train[f'Date_{fld}'] = getattr(train['Date'].dt, fld)
train.head()
train.drop(['Sales', 'Date'], axis=1, inplace=True)

In [41]:
import lightgbm as lgb

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [42]:
parameters = {
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_threads': 4,
    "metric": 'auc'
}
n_rounds = 10000

lgb_train = lgb.Dataset(train, label=y, free_raw_data=False, )

In [43]:
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
skf = TimeSeriesSplit(n_splits=3)

In [44]:
result = lgb.cv(parameters, lgb_train, n_rounds, 
                folds=skf.split(train, y), early_stopping_rounds=10, verbose_eval=1, 
                   )

[1]	cv_agg's auc: 0.999805 + 5.85438e-05
[2]	cv_agg's auc: 0.999805 + 5.84164e-05
[3]	cv_agg's auc: 0.999787 + 3.94029e-05
[4]	cv_agg's auc: 0.999787 + 3.91578e-05
[5]	cv_agg's auc: 0.999787 + 3.91651e-05
[6]	cv_agg's auc: 0.999787 + 3.9506e-05
[7]	cv_agg's auc: 0.999787 + 3.96921e-05
[8]	cv_agg's auc: 0.999786 + 3.93125e-05
[9]	cv_agg's auc: 0.999786 + 3.92448e-05
[10]	cv_agg's auc: 0.999786 + 3.92851e-05
[11]	cv_agg's auc: 0.999787 + 3.97161e-05


In [45]:
# TODO: CV & timeseries splitting do not work out of the box
# https://github.com/microsoft/LightGBM/pull/298/files

In [46]:
y.hist()